In [145]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import lightgbm as lgb

In [96]:
train = pd.read_csv("./final_train.csv")

In [150]:
x = train.drop("matched_score",axis=1)
y = train["matched_score"]

In [195]:
def high_corelated(data):
    correlation_matrix = data.corr().abs()
    # Select the upper triangle of the correlation matrix
    upper_triangle = correlation_matrix.where(
        np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
    )
    
    threshold = 0.8
    to_drop = [
        column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)
    ]
    
    # Drop highly correlated features
    data_reduced = data.drop(columns=to_drop)
    return data_reduced
x_reduced = high_corelated(x)

In [197]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.01)  # Adjust alpha for regularization strength
model.fit(x_reduced, y)
important_features = [feature for feature, coef in zip(x_reduced.columns, model.coef_) if coef != 0]
X_reduced = x_reduced[important_features]

In [200]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(x_train)
test_scaled = scaler.transform(x_test)

In [202]:
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)

# Train the model
rf_model.fit(train_scaled, y_train)

# Make predictions
y_pred = rf_model.predict(test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Mean Squared Error: 0.014337221747034702
R^2 Score: 0.4539353312802378


In [192]:
mse #raw data
# 0.011234893647222749

0.013590098515002988

In [213]:
params =  {'n_estimators': 466, 'learning_rate': 0.04203289188447668, 'num_leaves': 96, 'max_depth': 14, 'min_child_samples': 16, 'subsample': 0.9461135144192037, 'colsample_bytree': 0.64754815569208, 'reg_alpha': 0.42564312159285306, 'reg_lambda': 9.99144710065846}


lgb_model = lgb.LGBMRegressor(**params)

# Train the model
lgb_model.fit(
    train_scaled, y_train,
    eval_set=[(test_scaled, y_test)],
    eval_metric="mse",
)

# Validate the model
y_val_pred = lgb_model.predict(test_scaled)
mse = mean_squared_error(y_test, y_val_pred)
print(f"Validation MSE: {mse:.4f}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003434 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 240
[LightGBM] [Info] Number of data points in the train set: 6108, number of used features: 15
[LightGBM] [Info] Start training from score 0.658422
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

In [206]:
# Initialize the XGBoost model
xgb_model = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    random_state=42,
    verbosity=1
)

# Train the model
xgb_model.fit(
    train_scaled, y_train,
    eval_set=[(test_scaled, y_test)],
)


# Validate the model
y_val_pred = xgb_model.predict(test_scaled)
mse = mean_squared_error(y_test, y_val_pred)
print(f"Validation MSE: {mse}")

[0]	validation_0-rmse:0.15949
[1]	validation_0-rmse:0.15681
[2]	validation_0-rmse:0.15436
[3]	validation_0-rmse:0.15226
[4]	validation_0-rmse:0.15004
[5]	validation_0-rmse:0.14831
[6]	validation_0-rmse:0.14660
[7]	validation_0-rmse:0.14489
[8]	validation_0-rmse:0.14333
[9]	validation_0-rmse:0.14210
[10]	validation_0-rmse:0.14080
[11]	validation_0-rmse:0.13978
[12]	validation_0-rmse:0.13865
[13]	validation_0-rmse:0.13764
[14]	validation_0-rmse:0.13674
[15]	validation_0-rmse:0.13583
[16]	validation_0-rmse:0.13504
[17]	validation_0-rmse:0.13422
[18]	validation_0-rmse:0.13343
[19]	validation_0-rmse:0.13275
[20]	validation_0-rmse:0.13220
[21]	validation_0-rmse:0.13157
[22]	validation_0-rmse:0.13090
[23]	validation_0-rmse:0.13039
[24]	validation_0-rmse:0.12983
[25]	validation_0-rmse:0.12942
[26]	validation_0-rmse:0.12900
[27]	validation_0-rmse:0.12865
[28]	validation_0-rmse:0.12805
[29]	validation_0-rmse:0.12746
[30]	validation_0-rmse:0.12716
[31]	validation_0-rmse:0.12678
[32]	validation_0-

In [210]:
from catboost import CatBoostRegressor

In [212]:
catboost_model = CatBoostRegressor(
    iterations=500,          # Number of boosting iterations
    learning_rate=0.05,      # Learning rate
    depth=6,                 # Depth of the tree
    loss_function="RMSE",    # Loss function for regression
    random_state=42,
    verbose=100              # Print progress every 100 iterations
)

# Train the model
catboost_model.fit(
    train_scaled, y_train,
    eval_set=(test_scaled, y_test),
    early_stopping_rounds=50  # Stop training if no improvement in 50 iterations
)

# Validate the model
y_val_pred = catboost_model.predict(test_scaled)
mse = mean_squared_error(y_test, y_val_pred)
print(f"Validation MSE: {mse}")

0:	learn: 0.1662765	test: 0.1603434	best: 0.1603434 (0)	total: 3.56ms	remaining: 1.77s
100:	learn: 0.1192215	test: 0.1224002	best: 0.1224002 (100)	total: 359ms	remaining: 1.42s
200:	learn: 0.1115694	test: 0.1173992	best: 0.1173992 (200)	total: 650ms	remaining: 967ms
300:	learn: 0.1058620	test: 0.1138943	best: 0.1138943 (300)	total: 938ms	remaining: 620ms
400:	learn: 0.1018464	test: 0.1120289	best: 0.1120289 (400)	total: 1.24s	remaining: 305ms
499:	learn: 0.0986236	test: 0.1109068	best: 0.1109068 (499)	total: 1.53s	remaining: 0us

bestTest = 0.1109067779
bestIteration = 499

Validation MSE: 0.012300312448031167


In [ ]:
test_data = pd.read_csv("final_test.csv")

# Make predictions on the test set
test_predictions = model.predict(test_data)

# Load the sample submission file
sample_submission = pd.read_csv("sample_submission.csv")

# Create the submission file
submission = sample_submission.copy()
submission["matched_score"] = test_predictions
submission.to_csv("submission.csv", index=False)

print("Submission file created: submission.csv")